In [1]:
from msal import ConfidentialClientApplication
import pandas as pd
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import logging
import pandas as pd

def_credential = DefaultAzureCredential()  

## connect to Azure Key Vault securely
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"
kv_client = SecretClient(vault_url=KVUri, credential=def_credential)

## Secret Keys for Azure App Registration
tenant_id = kv_client.get_secret('azure-tenant-id').value
client_id = kv_client.get_secret('idgov-app-client-id').value
client_secret = kv_client.get_secret('idgov-app-client-secret').value

import sys
os.chdir('../')

In [2]:
from module.azure_ad import AzureAD

## Initialize AD and Bolddesk API Module
ad = AzureAD(tenant_id, client_id, client_secret)

from module.warehouse import Warehouse
## initialize warehouse
wh = Warehouse(
    server=os.environ["DB_SERVER"],
    database=os.environ["DB_NAME"],
    credential=def_credential
)

In [ ]:
# ## run the jobs
# from module.idgov     import save_to_warehouse, add_new_user_to_bd, deactivate_invalid_agent
# save_to_warehouse(ad, def_credential)

In [3]:
df = ad.list_one_drive_usage()
# table_name = 'ad_onedrive_usage'
# wh.refresh_table_rows(table_name, df, column_name='refresh_date', value=df.refresh_date[0])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 55 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   id                                         141 non-null    object
 1   userId                                     141 non-null    object
 2   deviceName                                 141 non-null    object
 3   managedDeviceOwnerType                     141 non-null    object
 4   enrolledDateTime                           141 non-null    object
 5   lastSyncDateTime                           141 non-null    object
 6   operatingSystem                            141 non-null    object
 7   complianceState                            141 non-null    object
 8   jailBroken                                 141 non-null    object
 9   managementAgent                            141 non-null    object
 10  osVersion                             

In [10]:
df = ad.list_managed_devices()
table_name = 'ad_managed_devices'
wh.erase(table_name)
wh.append(table_name, df)
